Day2.ipynb - We already have 800+ rows. Cleaning this data into useful insights will happen here.

In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [5]:
df = pd.read_csv("/home/kepha/nairobi_property/data/raw_listings.csv")
df_clean = df.copy()
print(f"Starting rows: {len(df_clean)}")

Starting rows: 808


REMOVING DUPLICATES

In [ ]:
print(f"Duplicates before: {df_clean.duplicated().sum()}")
df_clean = df_clean.drop_duplicates()

Duplicates before: 432


In [8]:
print(f"Rows after removing duplicates: {len(df_clean)}")
print(f"Duplicates after: {df_clean.duplicated().sum()}")

Rows after removing duplicates: 376
Duplicates after: 0


REMOVING PRICE OUTLIERS

In [26]:
df_clean = df_clean[(df_clean['price_kes'] >= 1_000_000) & 
                    (df_clean['price_kes'] <= 500_000_000)]
print(f"After price filtering, 1M-500M: {len(df_clean)} rows remain")


After price filtering, 1M-500M: 375 rows remain


SORTING SIZE OUTLIERS

In [27]:
print(df_clean['size_sqft'].describe())

count       352.000000
mean       6340.813750
std       18488.097679
min          96.880000
25%         839.590000
50%        1291.680000
75%        2454.190000
max      129170.000000
Name: size_sqft, dtype: float64


In [ ]:
# Identifying suspiciously small sizes, under 200 sqft but the price is over 10M KES
suspicious_small = (df_clean['size_sqft'] < 200) & (df_clean['price_kes'] > 10000000)
print(df_clean.loc[suspicious_small, ['location', 'bedrooms', 'size_sqft', 'price_kes']])


In [ ]:
#Multiply the size by 10 for these suspicious entries, assuming a possible decimal point error
df_clean.loc[suspicious_small, 'size_sqft'] = df_clean.loc[suspicious_small, 'size_sqft'] * 10
print(df_clean.loc[suspicious_small, ['location', 'bedrooms', 'size_sqft', 'price_kes']])


In [ ]:
# Add acres column to understand scale
df_clean['size_acres'] = df_clean['size_sqft'] / 43560

large_properties = df_clean[df_clean['size_sqft'] > 20000][
    ['location', 'bedrooms', 'size_sqft', 'size_acres', 'price_kes']
].sort_values('size_sqft', ascending=False)

print(large_properties)

In [31]:
# I have decided to remove the large properties that we multiplied by 10, as they are likely outliers and may skew the analysis. I will filter them out based on their size and price per sqft.
large_properties = df_clean[df_clean['size_sqft'] > 20000]
df_clean = df_clean[~df_clean.index.isin(large_properties.index)]

In [34]:
df_clean = df_clean[(df_clean['size_sqft'] >= 200) | (df_clean['size_sqft'].isna())]
print(f"After size filtering (>200 sqft): {len(df_clean)} rows")


After size filtering (>200 sqft): 351 rows


In [ ]:

missing_sizes = df_clean['size_sqft'].isna().sum()
print(f"Rows with missing sizes: {missing_sizes}")

Rows with missing sizes: 23


In [36]:
#DRopping rows with missing sizes for now, as we cannot impute them without more information
df_clean = df_clean.dropna(subset=['size_sqft'])
print(f"After dropping missing sizes: {len(df_clean)} rows")

After dropping missing sizes: 328 rows


SORTING AMMENITIES

In [38]:
#dropping properties with no ammenities listed, as they are likely incomplete listings
df_clean = df_clean.dropna(subset=['amenities'])
print(f"After dropping missing amenities: {len(df_clean)} rows")

After dropping missing amenities: 325 rows


FINALLY : STANDARDIZING THE TEXTS

In [40]:
df_clean['location'] = df_clean['location'].str.title().str.strip()
df_clean['property_type'] = df_clean['property_type'].str.title().str.strip()

In [43]:
print(f"Cleaning complete, Kept {len(df_clean)} listings")

Cleaning complete, Kept 325 listings


FEATURE ENGINEERING

In [44]:
# Price per square foot
df_clean['price_per_sqft'] = df_clean['price_kes'] / df_clean['size_sqft']
print("Created: price_per_sqft")

Created: price_per_sqft


In [46]:
#Creating amenity score
#Counts amenities each property has by counting commas eg "Parking, Pool, Gym" has 2 commas = 3 amenities.

def count_amenities(amenities_str):
    if pd.isna(amenities_str) or amenities_str == 'None':
        return 0
    return amenities_str.count(',') + 1

df_clean['amenity_score'] = df_clean['amenities'].apply(count_amenities)
print("Created: amenity_score")

Created: amenity_score


In [47]:
# Month from listing date
df_clean['listing_date'] = pd.to_datetime(df_clean['listing_date'])
df_clean['month'] = df_clean['listing_date'].dt.month
print("Created: month")

Created: month


In [51]:
df_clean.drop(columns=['size_acres'], inplace=True)

SAVE TO CLEAN_LISTINGS CSV

In [53]:
df_clean.to_csv("/home/kepha/nairobi_property/data/clean_listings.csv", index=False)